In [ ]:
!pip3 install --upgrade pip
!pip3 install boto3
!pip3 install pyyaml

In [1]:
from functools import reduce
import os
import boto3
from botocore.client import Config
import yaml
from datetime import datetime, time, timedelta
import json
from pprint import pprint
from pprint import PrettyPrinter
from statistics import mean
import sys

In [ ]:
print(sys.version_info)
print(sys.version_info[:3])

In [ ]:
config = yaml.load(open('config.yaml', 'r'))
#print(config)
aws_region = config['aws']['regions'][0]
account = config['aws']['accounts'][1]['name']
profile = config['aws']['accounts'][1]['profile']
account_number = config['aws']['accounts'][1]['account-number']

print(("%s - %s - %s - %s" % (str(account_number),account,profile,aws_region)))

In [ ]:
session = boto3.session.Session(profile_name=profile,region_name=aws_region)
print(session)

In [ ]:
ecsClient = session.client('ecs')
print(ecsClient)

In [ ]:
class EcsCluster(object):
    def __init__(self, clusterName, clusterArn, status, registeredContainerInstancesCount, runningTasksCount, pendingTasksCount, activeServicesCount):
        self.clusterName = clusterName
        self.clusterArn = clusterArn
        self.status = status
        self.registeredContainerInstancesCount = registeredContainerInstancesCount
        self.runningTasksCount = runningTasksCount
        self.pendingTasksCount = pendingTasksCount
        self.activeServicesCount = activeServicesCount
        self.ec2s = []
        
    def set_ec2s(self, ec2s):
        self.ec2s = ec2s
        
    def add_ec2s(self, ec2s):
        self.ec2s.extend(ec2s)
        
    def __str__(self):
        retrn = 'EcsCluster: clusterName={0}, clusterArn={1}, status={2}, registeredContainerInstancesCount={3}, runningTasksCount={4}, pendingTasksCount={5}, activeServicesCount={6}, ec2s={7}'.format(self.clusterName, self.clusterArn, self.status, str(self.registeredContainerInstancesCount), str(self.runningTasksCount), str(self.pendingTasksCount), str(self.activeServicesCount),tuple(self.ec2s))
        return retrn

In [ ]:
# Create a reusable Paginator
paginator = ecsClient.get_paginator('list_clusters')
#print(paginator)

# Create a PageIterator from the Paginator
page_iterator = paginator.paginate()

#clusterArns = []
clusters = []
#clusterEc2s = {}

for page in page_iterator:
    #print(page['clusterArns'])
    #clusterArns.extend(page['clusterArns'])
    
    response = ecsClient.describe_clusters(clusters=page['clusterArns'])
    #print(response)
    for cluster in response.get('clusters'):
        clusters.append(EcsCluster(cluster.get('clusterName'),cluster.get('clusterArn'),cluster.get('status'),cluster.get('registeredContainerInstancesCount'),cluster.get('runningTasksCount'),cluster.get('pendingTasksCount'),cluster.get('activeServicesCount')))
    #print(response.get(clusters))
    #print(page)

#print(len(clusterArns))
#print(clusterArns)
#print(len(clusters))


In [ ]:
#Long Running
for cluster in clusters:
    response = ecsClient.list_container_instances(cluster=cluster.clusterName)
    cluster.set_ec2s(response.get('containerInstanceArns'))
    #clusterEc2s[cluster.get('clusterName')] = response.get('containerInstanceArns')
    
#print(clusterEc2s)


In [ ]:
#pp = PrettyPrinter() #indent=1)
for cluster in clusters:
    #print(cluster)
    if cluster.registeredContainerInstancesCount > 10:
        pp.pprint(str(cluster))
    